In [1]:
# Celda 1
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Simple_Processing") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

print("✅ SparkSession creada")

✅ SparkSession creada


In [2]:
# Celda 2: Leer los datos 
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("encoding", "UTF-8") \
    .csv("Data/*.csv")

print("📚 Datos leídos exitosamente")

📚 Datos leídos exitosamente


In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

# Schema explícito basado en el diccionario de datos del MEF
mef_schema = StructType([
    StructField("ANO_EJE", IntegerType(), True),
    StructField("MES_EJE", IntegerType(), True),
    StructField("NIVEL_GOBIERNO", StringType(), True),
    StructField("NIVEL_GOBIERNO_NOMBRE", StringType(), True),
    StructField("SECTOR", StringType(), True),
    StructField("SECTOR_NOMBRE", StringType(), True),
    StructField("PLIEGO", StringType(), True),
    StructField("PLIEGO_NOMBRE", StringType(), True),
    StructField("SEC_EJEC", IntegerType(), True),
    StructField("EJECUTORA", IntegerType(), True),
    StructField("EJECUTORA_NOMBRE", StringType(), True),
    StructField("DEPARTAMENTO_EJECUTORA", IntegerType(), True),
    StructField("DEPARTAMENTO_EJECUTORA_NOMBRE", StringType(), True),
    StructField("PROVINCIA_EJECUTORA", IntegerType(), True),
    StructField("PROVINCIA_EJECUTORA_NOMBRE", StringType(), True),
    StructField("DISTRITO_EJECUTORA", IntegerType(), True),
    StructField("DISTRITO_EJECUTORA_NOMBRE", StringType(), True),
    StructField("SEC_FUNC", IntegerType(), True),
    StructField("PROGRAMA_PPTO", IntegerType(), True),
    StructField("PROGRAMA_PPTO_NOMBRE", StringType(), True),
    StructField("TIPO_ACT_PROY", IntegerType(), True),
    StructField("TIPO_ACT_PROY_NOMBRE", StringType(), True),
    StructField("PRODUCTO_PROYECTO", IntegerType(), True),
    StructField("PRODUCTO_PROYECTO_NOMBRE", StringType(), True),
    StructField("ACTIVIDAD_ACCION_OBRA", IntegerType(), True),
    StructField("ACTIVIDAD_ACCION_OBRA_NOMBRE", StringType(), True),
    StructField("FUNCION", IntegerType(), True),
    StructField("FUNCION_NOMBRE", StringType(), True),
    StructField("DIVISION_FUNCIONAL", IntegerType(), True),
    StructField("DIVISION_FUNCIONAL_NOMBRE", StringType(), True),
    StructField("GRUPO_FUNCIONAL", IntegerType(), True),
    StructField("GRUPO_FUNCIONAL_NOMBRE", StringType(), True),
    StructField("META", IntegerType(), True),
    StructField("FINALIDAD", IntegerType(), True),
    StructField("META_NOMBRE", StringType(), True),
    StructField("DEPARTAMENTO_META", IntegerType(), True),
    StructField("DEPARTAMENTO_META_NOMBRE", StringType(), True),
    StructField("FUENTE_FINANCIAMIENTO", IntegerType(), True),
    StructField("FUENTE_FINANCIAMIENTO_NOMBRE", StringType(), True),
    StructField("RUBRO", IntegerType(), True),
    StructField("RUBRO_NOMBRE", StringType(), True),
    StructField("TIPO_RECURSO", StringType(), True),
    StructField("TIPO_RECURSO_NOMBRE", StringType(), True),
    StructField("CATEGORIA_GASTO", IntegerType(), True),
    StructField("CATEGORIA_GASTO_NOMBRE", StringType(), True),
    StructField("TIPO_TRANSACCION", IntegerType(), True),
    StructField("GENERICA", IntegerType(), True),
    StructField("GENERICA_NOMBRE", StringType(), True),
    StructField("SUBGENERICA", IntegerType(), True),
    StructField("SUBGENERICA_NOMBRE", StringType(), True),
    StructField("SUBGENERICA_DET", IntegerType(), True),
    StructField("SUBGENERICA_DET_NOMBRE", StringType(), True),
    StructField("ESPECIFICA", IntegerType(), True),
    StructField("ESPECIFICA_NOMBRE", StringType(), True),
    StructField("ESPECIFICA_DET", IntegerType(), True),
    StructField("ESPECIFICA_DET_NOMBRE", StringType(), True),
    StructField("MONTO_PIA", DoubleType(), True),
    StructField("MONTO_PIM", DoubleType(), True),
    StructField("MONTO_CERTIFICADO", DoubleType(), True),
    StructField("MONTO_COMPROMETIDO_ANUAL", DoubleType(), True),
    StructField("MONTO_COMPROMETIDO", DoubleType(), True),
    StructField("MONTO_DEVENGADO", DoubleType(), True),
    StructField("MONTO_GIRADO", DoubleType(), True)
])

print("✅ Schema del MEF definido con 63 columnas")

✅ Schema del MEF definido con 63 columnas


In [5]:
# Validación contra metadata esperada del MEF
print("="*60)
print("VALIDACIÓN CONTRA METADATA MEF")
print("="*60)

# 1. Validar número de columnas
expected_columns = 63  # Según schema MEF
actual_columns = len(df.columns)
print(f"📋 Columnas esperadas: {expected_columns}")
print(f"📋 Columnas encontradas: {actual_columns}")
print(f"✅ Columnas: {'CORRECTO' if actual_columns == expected_columns else 'ERROR'}")

# 2. Validar presencia de columnas clave
key_columns = ["ANO_EJE", "MONTO_PIM", "MONTO_DEVENGADO", "PLIEGO", "SECTOR"]
missing_columns = [col for col in key_columns if col not in df.columns]
print(f"🔑 Columnas clave faltantes: {missing_columns if missing_columns else 'NINGUNA'}")

# 3. Validar años presentes
if "ANO_EJE" in df.columns:
    years = df.select("ANO_EJE").distinct().collect()
    year_list = [str(row['ANO_EJE']) for row in years]
    print(f"📅 Años encontrados: {', '.join(year_list)}")

VALIDACIÓN CONTRA METADATA MEF
📋 Columnas esperadas: 63
📋 Columnas encontradas: 63
✅ Columnas: CORRECTO
🔑 Columnas clave faltantes: NINGUNA
📅 Años encontrados: 2022, 2023, 2024, 2025


In [6]:
# Detección de registros corruptos/incompletos
from pyspark.sql.functions import col, sum as spark_sum

print("="*60)
print("DETECCIÓN DE REGISTROS CORRUPTOS")
print("="*60)

# 1. Registros con valores nulos en columnas críticas
critical_nulls = df.filter(
    col("ANO_EJE").isNull() | 
    col("MONTO_PIM").isNull() |
    col("PLIEGO").isNull()
).count()
print(f"🚫 Registros con nulos en columnas críticas: {critical_nulls}")

# 2. Porcentaje de completitud por columna
print("📊 Porcentaje de completitud por columna:")
completeness = df.select([
    (1 - (spark_sum(col(c).isNull().cast("int")) / df.count())).alias(c) 
    for c in df.columns[::10]  # Cada 10 columnas para no saturar
])
completeness.show(vertical=True)

DETECCIÓN DE REGISTROS CORRUPTOS
🚫 Registros con nulos en columnas críticas: 0
📊 Porcentaje de completitud por columna:
-RECORD 0-----------------
 ANO_EJE            | 1.0 
 EJECUTORA_NOMBRE   | 1.0 
 TIPO_ACT_PROY      | 1.0 
 GRUPO_FUNCIONAL    | 1.0 
 RUBRO_NOMBRE       | 1.0 
 SUBGENERICA_DET    | 1.0 
 MONTO_COMPROMETIDO | 1.0 



In [7]:
# Eliminar duplicados exactos
initial_count = df.count()
df_clean = df.dropDuplicates()
final_count = df_clean.count()
duplicates_removed = initial_count - final_count

print("="*60)
print("ELIMINACIÓN DE DUPLICADOS")
print("="*60)
print(f"📊 Registros iniciales: {initial_count:,}")
print(f"🧹 Duplicados eliminados: {duplicates_removed:,}")
print(f"✅ Registros finales: {final_count:,}")
print(f"📉 Reducción: {(duplicates_removed/initial_count*100):.2f}%")

ELIMINACIÓN DE DUPLICADOS
📊 Registros iniciales: 40,173,905
🧹 Duplicados eliminados: 13
✅ Registros finales: 40,173,892
📉 Reducción: 0.00%


In [8]:
# Guardar particionado por año con estructura MEF
df_clean.write \
    .mode("overwrite") \
    .partitionBy("ANO_EJE") \
    .parquet("data/raw/year")

print("="*60)
print("ALMACENAMIENTO COMPLETADO")
print("="*60)
print("💾 Datos guardados en formato Parquet")
print("🗂️  Estructura: data/raw/year=ANO_EJE/")
print("✅ Formato: Particionado por año (como solicita MEF)")
print("📋 Schema: Definición explícita según diccionario MEF")

ALMACENAMIENTO COMPLETADO
💾 Datos guardados en formato Parquet
🗂️  Estructura: data/raw/year=ANO_EJE/
✅ Formato: Particionado por año (como solicita MEF)
📋 Schema: Definición explícita según diccionario MEF


In [9]:
# Verificación final
print("="*60)
print("VERIFICACIÓN FINAL FASE 1")
print("="*60)

# Leer una partición para verificar
df_verify = spark.read.parquet("data/raw/year")
print(f"✅ Registros verificados: {df_verify.count():,}")
print(f"✅ Particiones: {len(df_verify.inputFiles())} archivos")
print(f"✅ Columnas: {len(df_verify.columns)}")
print("🎉 FASE 1 COMPLETADA EXITOSAMENTE")

VERIFICACIÓN FINAL FASE 1
✅ Registros verificados: 40,173,892
✅ Particiones: 800 archivos
✅ Columnas: 63
🎉 FASE 1 COMPLETADA EXITOSAMENTE
